* Base LLM: predicts next word.
* Instruction Tuned LLM: follow instructions.

In [1]:
!pip install openai

     ---------------------------------------- 71.9/71.9 kB 3.9 MB/s eta 0:00:00
     ------------------------------------- 324.5/324.5 kB 10.1 MB/s eta 0:00:00
     ---------------------------------------- 61.8/61.8 kB 3.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## Principles and Tactics of Prompting
### Write clear and specific instructions.
1. Use delimiters:
    * Triple quotes: """
    * Triple backticks: ```
    * Triple dashes: ---
    * Angle brackets: <>
    * XML tags: `<tag> </tag>`

```
Summarize the text delimited by triple backticks into a single sentence.
```

2. Ask for a structured output, e.g., JSON or HTML. 

```
Generate a list of xx. Provide them in JSON format with the following keys: book_id, title, author, genre.
```

3. Ask the model to check whether conditions are satisfied. 

```
If it contains a sequence of instructions, re-write those instructions in the following format: Step 1 - ...; If the text does not contain a sequence of instructions, then simply write \"No steps provided.\"
```

4. "Few-shot" prompting: Give successful examples of completing tasks. 

```
Your task is to answer in a consistent style. <Question1>; <Answer1>; <Question2>.
```

### Give the model time to think.
1. Specify the steps to complete a task. 

```
Perform the following actions: 1. Summarize the following text. 2. Translate the summary into French. 3. List each name in the French summary. 4. Output a json object that contains the following keys: french_summary, num_names.
```
2. Instruct the model to work out its own solution before rushing to a conclusion. 

```
Use the following format: <Question>; <Student's solution>; <Actual solution>; <Is the student's solution the same as actual solution just calculated: yes or no>; <Student grade: correct or incorrect>
```

## Iterative Prompt Develelopment


Keep looping the following procedures.
* Implementation (code/data) Prompt: Try something
* Experimental result
* Error Analysis: Analyze where the results does not give what you want.
* Idea

* Ask GPT to limit the number of words/sentences/characters.

```
Use at most 50 words.
Use at most three sentences.
Use at most 280 characters.
```

* Ask GPT to focus on the right details.

```
The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.
```

* Ask it to extract information and organize it in a table.

```
After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.
```

# Applications

## Summarizing
* News
```
Your task is to generate a short summary of a product \
review from an ecommerce site. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words.
```
* Summarizing with focus
```
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
Shipping deparmtment. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects \
that mention shipping and delivery of the product.
```

```
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
pricing deparmtment, responsible for determining the \
price of the product.  

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects \
that are relevant to the price and perceived value.
```

* Try "extract" instead of "summarize"
```
Your task is to extract relevant information from \ 
a product review from an ecommerce site to give \
feedback to the Shipping department. 

From the review below, delimited by triple quotes \
extract the information relevant to shipping and \ 
delivery. Limit to 30 words. 
```
* Summarize from multiple sources
* Report
* Product description
```
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.
```

## Extracting

* Extract product and company name from customer reviews
```
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
```

## Inferrring

* Sentiment (positive/negative)
```
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".
```

* Identify types of emotions
```
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.
```

* Indentify anger.
```
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.
```

* Infer 5 topics
```
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.
```

* Make a news alert for certain topics
```
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\
```

## Others
* Doing multiple tasks at once
```
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.
```

## Transforming

* language translation
```
Translate the following English text to Spanish
```

* spelling and grammar checking
```
Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found".
```
* tone adjustment
```
Translate the following from slang to a business letter: 
'Dude, This is Joe, check out this spec on this standing lamp.'
```

* format conversion: especially useful in coding
```{python}
data_json = { "resturant employees" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""
```

## Expanding
* Customize the automated reply to a customer email
```
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
```

## ChatBot

In [ ]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

### Data Process
* Change raw data into the json format.
The text delimited by triple backticks is the data, every row represents the key and the value, and the key and the value are seperated by colon. Do not show me the code. Help me transfer the text to the json format.

### Writing
* plural and 拼写错误

* Find a better expression from Chinese to English. This function can be combined with the zotero search of attachment content.
```
How should I expressed the text delimited by triple backticks in english? Give me three sentences  as examples. This is a translation task.
```
* Academic Style:
```
Rewrite the text delimited by triple backticks in clear and in academic language.
```
* Grammar Check:
```
Please make suggestions for fixing any language issues in the text delimited by triple backticks and provide an explanation for each suggested change.
```
* Report Generation:

### Coding
* Visualization

However, the model has a limitation called **hallucinations**, i.e., it may make statements that sound plausible but are not true. One way to reduce hallucinations: `First find relevant information, then answer the question based on the relevant information.`

In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]


# Possible Issues
* Unstable results